In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Input
from keras.layers import concatenate
from keras.optimizers import SGD
from keras.layers.core import Lambda
import keras.backend as K


Using TensorFlow backend.


In [3]:
# model parameter
# SGD part
SGD_LSTM_OUTPUT = 4
SGD_W_LAYER_1 = 64
SGD_W_LAYER_2 = 64
SGD_W_OUTPUT = 1

# Conditional part
CON_LSTM_OUTPUT = 4
CON_G_OUTPUT = 4

# learning parameter
learning_rate = 0.001
dropout_size = 0.95

timesteps = 100 # days
data_dim = 100 # features

# output
output = np.zeros((1000, 50*8))

# SGD 神经网络

其中包括 LSTM，输出为4个状态，再加入一个64-64-1的DNN，然后构建$M_{t+1}R_{t+1}$，结合给定的$g$函数，计算均方误差

In [4]:
# 考虑1000天，50只股票，100个微观因子，10个宏观因子
# SGD networks
macro_in1 = Input(shape=(10,))
macro_in1_1 = Lambda(lambda x:K.reshape(x, shape=(-1,1, 10)))(macro_in1)


sgd_lstm_out = LSTM(4)(macro_in1_1) # (1000, 4)
sgd_lstm_out = Lambda(lambda x:K.repeat(x, 50))(sgd_lstm_out)

micro_in1 = Input(shape=(50,100)) 
return_next_date1 = Input(shape=(50,))

sgd_weights_input = concatenate([micro_in1, sgd_lstm_out]) # (1000, 50, 104)  

sgd_weights_output = Dense(64, activation='relu')(sgd_weights_input)
sgd_weights_output = Dense(64, activation='relu')(sgd_weights_input)
sgd_weights_output = Dense(1)(sgd_weights_input)
sgd_weights_output = Lambda(lambda x:K.reshape(x, shape=(-1, 50)))(sgd_weights_output)


def construction(x):
    tmp = x[0] * x[1]
    tmp = 1-K.sum(tmp, axis=1)
    tmp = K.reshape(tmp, shape=(-1,1)) # (1000, 1)
    tmp = K.repeat(tmp, 50) # (1000, 50, 1)
    tmp = K.reshape(tmp, shape=(-1,50)) # (1000, 50)
    tmp = tmp * x[1] # (1000, 50)
    tmp = K.reshape(tmp, shape=(-1, 50, 1))
    return tmp # the M_{t+1}R_{t+1}

sgd_construction_ouput = Lambda(construction)([sgd_weights_output,return_next_date1])


## attain con_g_output 
con_g_output_loaded = Input(shape=(50, 8))

loss_function_w = Lambda(lambda x:x[0]*x[1])([sgd_construction_ouput, con_g_output_loaded]) # (1000,50, 8)
loss_function_w = Lambda(lambda x:K.reshape(x, shape=(-1, 400)))(loss_function_w)

model_output_w = Model(inputs=[macro_in1, micro_in1], outputs=sgd_weights_output) # acquires weights given info

model_output_sgd = Model(inputs=[macro_in1, micro_in1, return_next_date1], outputs=sgd_construction_ouput) # acquires MR for condition networks

model_w = Model(inputs=[macro_in1, micro_in1, return_next_date1, con_g_output_loaded], outputs=loss_function_w)
model_w.compile(optimizer='adam', loss='mean_squared_error')


In [10]:
model_output_w.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 1, 10)        0           input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 4)            240         lambda_1[0][0]                   
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 50, 100)      0                                            
____________________________________________________________________________________________

In [6]:
model_w.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 1, 10)        0           input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 4)            240         lambda_1[0][0]                   
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 50, 100)      0                                            
____________________________________________________________________________________________

# 条件神经网络

其中包括 LSTM，输出为4个状态，再加入一个64-8的DNN，然后构建$g$，结合给定的$M_{t+1}R_{t+1}$，计算均方误差

In [7]:
# conditional networks
macro_in2 = Input(shape=(10,))
macro_in2_1 = Lambda(lambda x:K.reshape(x, shape=(-1,1, 10)))(macro_in2)

micro_in2 = Input(shape=(50,100))

return_next_date2 = Input(shape=(50,))

sgd_construction_ouput_loaded = Input(shape=(50,1))

con_lstm_out = LSTM(4)(macro_in2_1) # (1000, 4)
con_lstm_out = Lambda(lambda x:K.repeat(x, 50))(con_lstm_out)

con_weights_input = concatenate([micro_in2, con_lstm_out]) # (1000, 50, 104)  

con_g_output = Dense(64, activation='relu')(con_weights_input)
con_g_output = Dense(8)(con_g_output) # (1000, 50, 8)

loss_function_g = Lambda(lambda x:1/(0.001+x[0]*x[1]))([sgd_construction_ouput_loaded, con_g_output]) # (1000,50, 8)
loss_function_g = Lambda(lambda x:K.reshape(x, shape=(-1, 400)))(loss_function_g)

model_output_g = Model(inputs=[macro_in2, micro_in2], outputs=con_g_output) # acquires MR for condition networks

model_g = Model(inputs=[macro_in2, micro_in2, return_next_date2, sgd_construction_ouput_loaded], outputs=loss_function_g)
model_g.compile(optimizer='adam', loss='mean_squared_error')

In [8]:
model_g.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
lambda_7 (Lambda)               (None, 1, 10)        0           input_5[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, 4)            240         lambda_7[0][0]                   
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 50, 100)      0                                            
____________________________________________________________________________________________

# 训练

In [21]:
y_train = np.random.random((1000, 400))
macro_in = np.random.random((1000, 10))
micro_in = np.random.random((1000, 50, 100))
return_next_date = np.random.random((1000, 50))
function_g = np.random.random((1000, 50, 8))
for i in range(100):
    # SDF nets
    model_w.fit([macro_in, micro_in, return_next_date, function_g], y_train, epochs=10, batch_size=200)
    function_sgd = model_output_sgd.predict([macro_in, micro_in, return_next_date])
    print(function_sgd)
    # conditional nets
    model_g.fit([macro_in, micro_in, return_next_date, function_sgd], y_train, epochs=10, batch_size=200)
    function_g = model_output_g.predict([macro_in, micro_in])
    
# get the optimal weights(SDF weights)
print(model_output_w.predict([macro_in, micro_in])[0])

Epoch 1/10
1000/1000 [==============================] - 0s 89us/step - loss: 0.6718
Epoch 2/10
1000/1000 [==============================] - 0s 91us/step - loss: 0.5432
Epoch 3/10
1000/1000 [==============================] - 0s 76us/step - loss: 0.5234
Epoch 4/10
1000/1000 [==============================] - 0s 89us/step - loss: 0.5420
Epoch 5/10
1000/1000 [==============================] - 0s 100us/step - loss: 0.5214
Epoch 6/10
1000/1000 [==============================] - ETA: 0s - loss: 0.512 - 0s 84us/step - loss: 0.5116
Epoch 7/10
1000/1000 [==============================] - 0s 87us/step - loss: 0.5159
Epoch 8/10
1000/1000 [==============================] - 0s 76us/step - loss: 0.5136
Epoch 9/10
1000/1000 [==============================] - 0s 92us/step - loss: 0.5098
Epoch 10/10
1000/1000 [==============================] - 0s 88us/step - loss: 0.5113
[[[ 0.4875291 ]
  [ 2.1128895 ]
  [ 1.2352359 ]
  ...
  [ 2.1386821 ]
  [ 2.5825124 ]
  [ 0.72914296]]

 [[ 0.1278886 ]
  [ 0.11287855

1000/1000 [==============================] - 0s 129us/step - loss: 5394967.1000
Epoch 2/10
1000/1000 [==============================] - 0s 119us/step - loss: 2331322429.0000
Epoch 3/10
1000/1000 [==============================] - 0s 132us/step - loss: 20496495014117.1875
Epoch 4/10
1000/1000 [==============================] - 0s 108us/step - loss: 1000711911.3000
Epoch 5/10
1000/1000 [==============================] - 0s 132us/step - loss: 96130544.5250
Epoch 6/10
1000/1000 [==============================] - 0s 135us/step - loss: 395819190.4000
Epoch 7/10
1000/1000 [==============================] - 0s 133us/step - loss: 125074714.7000
Epoch 8/10
1000/1000 [==============================] - 0s 134us/step - loss: 4114387.4250
Epoch 9/10
1000/1000 [==============================] - 0s 137us/step - loss: 33544461.1250
Epoch 10/10
1000/1000 [==============================] - 0s 143us/step - loss: 189413517.5625
Epoch 1/10
1000/1000 [==============================] - 0s 141us/step - loss: 2

Epoch 2/10
1000/1000 [==============================] - 0s 123us/step - loss: 2.6432
Epoch 3/10
1000/1000 [==============================] - 0s 130us/step - loss: 2.6236
Epoch 4/10
1000/1000 [==============================] - 0s 110us/step - loss: 2.5741
Epoch 5/10
1000/1000 [==============================] - 0s 117us/step - loss: 2.5698
Epoch 6/10
1000/1000 [==============================] - 0s 123us/step - loss: 2.5657
Epoch 7/10
1000/1000 [==============================] - 0s 135us/step - loss: 2.5372
Epoch 8/10
1000/1000 [==============================] - 0s 137us/step - loss: 2.5578
Epoch 9/10
1000/1000 [==============================] - 2s 2ms/step - loss: 2.5305
Epoch 10/10
1000/1000 [==============================] - 0s 147us/step - loss: 2.5743
[[[ 0.30348182]
  [ 1.315252  ]
  [ 0.7689216 ]
  ...
  [ 1.3313075 ]
  [ 1.6075871 ]
  [ 0.45388395]]

 [[-0.11370055]
  [-0.10035573]
  [-0.2947744 ]
  ...
  [-0.38712788]
  [-0.25639385]
  [-0.02412229]]

 [[ 0.01630301]
  [ 0.055996

1000/1000 [==============================] - 0s 99us/step - loss: 31974297.3250
Epoch 2/10
1000/1000 [==============================] - 0s 108us/step - loss: 39580326.2250
Epoch 3/10
1000/1000 [==============================] - 0s 100us/step - loss: 18760673.8000
Epoch 4/10
1000/1000 [==============================] - 0s 114us/step - loss: 12199613.2000
Epoch 5/10
1000/1000 [==============================] - 0s 98us/step - loss: 12326320.4000
Epoch 6/10
1000/1000 [==============================] - 0s 122us/step - loss: 16931962.6000
Epoch 7/10
1000/1000 [==============================] - 0s 93us/step - loss: 68376708.1500
Epoch 8/10
1000/1000 [==============================] - 0s 108us/step - loss: 96521205.6000
Epoch 9/10
1000/1000 [==============================] - 0s 116us/step - loss: 4860932536.1750
Epoch 10/10
1000/1000 [==============================] - 0s 111us/step - loss: 199143830.1500
Epoch 1/10
1000/1000 [==============================] - 0s 139us/step - loss: 2.3436
Epoch

Epoch 2/10
1000/1000 [==============================] - 0s 104us/step - loss: 2.0591
Epoch 3/10
1000/1000 [==============================] - 0s 112us/step - loss: 2.0340
Epoch 4/10
1000/1000 [==============================] - 0s 96us/step - loss: 2.0284
Epoch 5/10
1000/1000 [==============================] - 0s 124us/step - loss: 2.0473
Epoch 6/10
1000/1000 [==============================] - 0s 109us/step - loss: 1.9969
Epoch 7/10
1000/1000 [==============================] - 0s 116us/step - loss: 1.9931
Epoch 8/10
1000/1000 [==============================] - 0s 118us/step - loss: 1.9734
Epoch 9/10
1000/1000 [==============================] - 0s 108us/step - loss: 1.9756
Epoch 10/10
1000/1000 [==============================] - 0s 119us/step - loss: 1.9981
[[[ 0.26214516]
  [ 1.1361041 ]
  [ 0.6641883 ]
  ...
  [ 1.1499727 ]
  [ 1.3886209 ]
  [ 0.39206132]]

 [[-0.12072586]
  [-0.10655648]
  [-0.31298783]
  ...
  [-0.4110476 ]
  [-0.27223584]
  [-0.02561275]]

 [[ 0.01108512]
  [ 0.03807

...

1000/1000 [==============================] - 0s 122us/step - loss: 419741811.2000
Epoch 2/10
1000/1000 [==============================] - 0s 103us/step - loss: 117683764.0000
Epoch 3/10
1000/1000 [==============================] - 0s 123us/step - loss: 120240704.4000
Epoch 4/10
1000/1000 [==============================] - 0s 118us/step - loss: 118222642.4000
Epoch 5/10
1000/1000 [==============================] - 0s 104us/step - loss: 117045448.8000
Epoch 6/10
1000/1000 [==============================] - 0s 122us/step - loss: 153755456.0000
Epoch 7/10
1000/1000 [==============================] - 0s 114us/step - loss: 120186775.6000
Epoch 8/10
1000/1000 [==============================] - 0s 106us/step - loss: 117505726.4000
Epoch 9/10
1000/1000 [==============================] - 0s 122us/step - loss: 116899312.6000
Epoch 10/10
1000/1000 [==============================] - 0s 114us/step - loss: 116324814.0000
Epoch 1/10
1000/1000 [==============================] - 0s 96us/step - loss: 0.3

1000/1000 [==============================] - 0s 118us/step - loss: 2173952808.4000
Epoch 2/10
1000/1000 [==============================] - 0s 106us/step - loss: 236624745.6000
Epoch 3/10
1000/1000 [==============================] - 0s 120us/step - loss: 200939141.6000
Epoch 4/10
1000/1000 [==============================] - 0s 116us/step - loss: 179266340.4000
Epoch 5/10
1000/1000 [==============================] - 0s 104us/step - loss: 167277984.4000
Epoch 6/10
1000/1000 [==============================] - 0s 122us/step - loss: 161259505.6000
Epoch 7/10
1000/1000 [==============================] - 0s 121us/step - loss: 156708202.9000
Epoch 8/10
1000/1000 [==============================] - 1s 1ms/step - loss: 150258939.2000
Epoch 9/10
1000/1000 [==============================] - 0s 136us/step - loss: 149017533.6000
Epoch 10/10
1000/1000 [==============================] - 0s 104us/step - loss: 147093504.0000
Epoch 1/10
1000/1000 [==============================] - 0s 90us/step - loss: 0.34

1000/1000 [==============================] - 0s 117us/step - loss: 0.3371
[[[ 0.03382551]
  [ 0.14659548]
  [ 0.08570254]
  ...
  [ 0.148385  ]
  [ 0.17917861]
  [ 0.05058904]]

 [[ 0.0319198 ]
  [ 0.02817343]
  [ 0.08275368]
  ...
  [ 0.10868058]
  [ 0.07197889]
  [ 0.00677199]]

 [[ 0.00319782]
  [ 0.01098362]
  [ 0.01270933]
  ...
  [ 0.00612361]
  [ 0.00825976]
  [ 0.00623648]]

 ...

 [[ 0.00832396]
  [ 0.00460442]
  [ 0.01368814]
  ...
  [ 0.00880286]
  [ 0.00113988]
  [ 0.0109361 ]]

 [[ 0.05198666]
  [ 0.05132309]
  [ 0.02185151]
  ...
  [ 0.06096926]
  [ 0.04397248]
  [ 0.04123629]]

 [[-0.00203396]
  [-0.00334087]
  [-0.00335842]
  ...
  [-0.0046698 ]
  [-0.00420611]
  [-0.00503179]]]
Epoch 1/10
1000/1000 [==============================] - 0s 126us/step - loss: 33779179057.6000
Epoch 2/10
1000/1000 [==============================] - 0s 126us/step - loss: 194181032.7000
Epoch 3/10
1000/1000 [==============================] - 0s 134us/step - loss: 88796696.2000
Epoch 4/10
1000/

1000/1000 [==============================] - 0s 143us/step - loss: 0.3435
Epoch 4/10
1000/1000 [==============================] - 0s 139us/step - loss: 0.3599
Epoch 5/10
1000/1000 [==============================] - 0s 130us/step - loss: 0.3468
Epoch 6/10
1000/1000 [==============================] - 0s 130us/step - loss: 0.3436
Epoch 7/10
1000/1000 [==============================] - 0s 119us/step - loss: 0.3367
Epoch 8/10
1000/1000 [==============================] - 0s 114us/step - loss: 0.3568
Epoch 9/10
1000/1000 [==============================] - 0s 125us/step - loss: 0.3376
Epoch 10/10
1000/1000 [==============================] - 0s 146us/step - loss: 0.3353
[[[ 0.02597483]
  [ 0.11257166]
  [ 0.06581156]
  ...
  [ 0.11394584]
  [ 0.13759246]
  [ 0.03884767]]

 [[ 0.03604475]
  [ 0.03181425]
  [ 0.09344783]
  ...
  [ 0.12272525]
  [ 0.08128063]
  [ 0.00764712]]

 [[ 0.00325995]
  [ 0.01119702]
  [ 0.01295626]
  ...
  [ 0.00624258]
  [ 0.00842024]
  [ 0.00635765]]

 ...

 [[ 0.015546

1000/1000 [==============================] - 0s 141us/step - loss: 237421940.0000
Epoch 2/10
1000/1000 [==============================] - 0s 150us/step - loss: 184576060.8000
Epoch 3/10
1000/1000 [==============================] - 0s 151us/step - loss: 145020454.9000
Epoch 4/10
1000/1000 [==============================] - 0s 168us/step - loss: 132510291.8000
Epoch 5/10
1000/1000 [==============================] - 0s 166us/step - loss: 120714872.0000
Epoch 6/10
1000/1000 [==============================] - 0s 179us/step - loss: 111830535.4000
Epoch 7/10
1000/1000 [==============================] - 0s 192us/step - loss: 107047419.6000
Epoch 8/10
1000/1000 [==============================] - 0s 163us/step - loss: 104157385.7000
Epoch 9/10
1000/1000 [==============================] - 0s 185us/step - loss: 100537694.8000
Epoch 10/10
1000/1000 [==============================] - 0s 152us/step - loss: 98112774.0000
Epoch 1/10
1000/1000 [==============================] - 0s 157us/step - loss: 0.3

In [20]:
K=model_output_w.predict([macro_in, micro_in])
K

array([[ 0.3191094 ,  0.37452483, -0.22857203, ..., -0.41956928,
        -0.2707104 ,  0.08568493],
       [-0.660378  , -0.07371467,  0.04483914, ...,  0.05806937,
        -0.3576033 ,  0.33140838],
       [-0.00507295, -0.2541618 , -0.4716102 , ..., -0.13226077,
         0.47486013, -0.48389784],
       ...,
       [ 0.55350894,  0.2512557 ,  0.10812303, ..., -0.36306193,
         0.26671496, -0.04334852],
       [ 0.5131978 , -0.41224778, -0.0729791 , ..., -0.14771284,
        -0.33065024, -0.14439678],
       [-0.03693056,  0.6412104 , -0.8023758 , ...,  0.2906107 ,
        -0.6937762 ,  0.24122457]], dtype=float32)

In [18]:
K.shape

(50,)